In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats, linalg
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rcParams

In [ ]:
rcParams.update({'legend.fontsize': 'x-large',
                 'figure.figsize': (12, 6),
                 'axes.labelsize': 'xx-large',
                 'axes.titlesize':'x-large',
                 'xtick.labelsize':'x-large',
                 'ytick.labelsize':'x-large'})

# P2 - Channel Equalization

This exercise aims at implementing a Wiener filter (with a varying number of coefficients) to equalize the transmission channel.

In [ ]:
beta = np.sqrt(0.27)

## a)

Implement the channel (neglect the noise) by defining a function `channel(input_stream, coefficients)` which describes the second order AR process.
The internal states of the memory cells can be initialized with zeros.

In [ ]:
def channel(input_stream, coefficients):
    """
    :param input_stream: Array of scaled BPSK signal beate * d(n).
    :param coefficients: Coefficients of the transfer function of the channel.
    """
    raise NotImplementedError

Test your channel function a little bit:

In [ ]:
expected_output = np.asarray([1., 3., 7., 14.])
output = channel([1, 2, 3, 4], [1, 1, 1])
np.testing.assert_equal(output, expected_output)

## b)

Calculate the autocorrelation matrix $\mathbf R_{\alpha}$ and the cross correlation vector $\mathbf p_{\mathbf xd}$ for a Wiener filter with 3 coefficients.
The input sequence is of variance $\sigma_d^2 = 1$.

Hint: Check [`scipy.linalg.toeplitz`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.toeplitz.html).

In [ ]:
def get_R_alpha(num_coefficients, beta=np.sqrt(.27), channel_coefficients=[1, .1, .8], noise_variance=1.):
    raise NotImplementedError

def get_crosscorrelation_vector(num_coefficients, beta=np.sqrt(.27), channel_coefficients=[1, .1, .8], noise_variance=1.):
    raise NotImplementedError

In [ ]:
get_R_alpha(3)

In [ ]:
get_crosscorrelation_vector(3)

## c)

Solve the Wiener-Hopf equation to estimate $d(n)$ with a Wiener filter having 3 coefficients.
Consider the noise-free and the noisy case ($\sigma_v^2 = 0.1$).

In [ ]:
def get_R_v(num_coefficients, noise_variance=0.1):
    raise NotImplementedError

def get_wiener_filter(num_coefficients, noise_variance=0.1):
    raise NotImplementedError

In [ ]:
get_wiener_filter(3, noise_variance=0)

In [ ]:
get_wiener_filter(3, noise_variance=.1)

## d)

Implement the application of the Wiener filter to the input sequence $d(n)$.

Hint: Check [`numpy.convolve`](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

In [ ]:
def filter_sequence(x, wf_coefficients):
    raise NotImplementedError

In [ ]:
d = np.asarray([1, -1, -1, 1, -1])
x = channel(beta * d, [1, .1, .8])
wf = get_wiener_filter(3, noise_variance=0)
d_hat = filter_sequence(x, wf)
print(f'Send: {d}, Recieved: {x}')
print(f'After channel equalization: {d_hat}')

## e)

Apply the Wiener filter to a white input sequence of $N=200$ BPSK-symbols transmitted over
the channel ($d(n) \in\{+1,-1\}$). Again, consider $v(n)=0$ and $v(n)\neq0$.
Compute the mean squared error for a wiener filter with 2, 3, and 4 coefficients, using $200$ 
observations for $v(n)=0$ and $v(n)\neq0$. Visualize  the error function for a Wiener filter with 2 coefficients.

In [ ]:
def apply_wiener_filter(N, noise_variance, num_coefficients, var_d=1):
    def mse(d, d_hat):
        raise NotImplementedError
        
    channel_coefficients = [1, 0.1, 0.8]
    
    wf = get_wiener_filter(num_coefficients, noise_variance)
    
    d = np.random.choice((-1, 1), N)
    
    alpha = channel(beta * d, channel_coefficients)
    v = np.random.normal(scale=np.sqrt(noise_variance), size=alpha.shape)
    x = alpha + v
    d_hat = filter_sequence(x, wf)
    
    print('Optimal coefficients: ', wf)
    
    # Analytical MSE
    j_mse = ???
    print('Analytic MSE-Error: ', j_mse)
    
    # Estimated MSE
    j_mse_hat = mse(d, d_hat)
    print('Estimated MSE-Error: ' ,j_mse_hat)

$v(n)=0$:

In [ ]:
apply_wiener_filter(N=200, noise_variance=0, num_coefficients=2)

In [ ]:
apply_wiener_filter(N=200, noise_variance=0, num_coefficients=3)

In [ ]:
apply_wiener_filter(N=200, noise_variance=0, num_coefficients=4)

$v(n) \neq 0$:

In [ ]:
apply_wiener_filter(N=200, noise_variance=.1, num_coefficients=2)

In [ ]:
apply_wiener_filter(N=200, noise_variance=.1, num_coefficients=3)

In [ ]:
apply_wiener_filter(N=200, noise_variance=.1, num_coefficients=4)

Error plane (for 2 coefficients):

In [ ]:
p_xd = get_crosscorrelation_vector(2)
r_x = get_R_alpha(2)[0]

x = np.arange(-2, 2, 0.1)
y = np.arange(-2, 2, 0.1)
xx, yy = np.meshgrid(x, y, sparse=True)

j_mse = 1 - 2 * (xx * p_xd[0] + yy * p_xd[1]) + r_x[0] * (xx ** 2 + yy ** 2) + 2 * r_x[1] * xx * yy
plt.contourf(x, y, j_mse, cmap=cm.viridis)
plt.colorbar()
plt.xlabel('$w_o(0)$')
plt.ylabel('$w_o(1)$')
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.gca(projection='3d')
ax.plot_surface(
    xx, yy, j_mse, cmap=cm.viridis, linewidth=1., rstride=4, cstride=4, edgecolor='w', antialiased=True
)
ax.set_xlabel('\n $w_o(0)$')
ax.set_ylabel('\n $w_o(1)$')
ax.set_zlabel('E$[|e(n)|^2]$')
m = cm.ScalarMappable(cmap=cm.viridis)
m.set_array(j_mse)
plt.colorbar(m)
plt.show()